In [101]:
from pyspark.sql import SparkSession
import os
import pyspark.sql.functions as F
spark = (
    SparkSession
    .builder
    .master("local[*]")
    .getOrCreate()
)
df = spark.read.parquet(
    os.path.join("/opt/workspace", "parquet")
).select(
    F.col("id"),
    F.col("name"),
    F.col("genres"),
    F.col("runtime"),
    F.col("status"),
    F.col("officialSite"),
)
df.show(5)

+---+------------------+--------------------+-------+-------+--------------------+
| id|              name|              genres|runtime| status|        officialSite|
+---+------------------+--------------------+-------+-------+--------------------+
|127|        The Affair|    [Drama, Romance]|     60|Running|http://www.sho.co...|
|128|   Jane the Virgin|[Drama, Comedy, R...|     60|Running|http://www.cwtv.c...|
|129|          Marry Me|            [Comedy]|     30|  Ended|                NULL|
|130|Two and a Half Men|            [Comedy]|     30|  Ended|http://www.cbs.co...|
|131|       About a Boy|[Drama, Comedy, R...|     30|  Ended|http://www.nbc.co...|
+---+------------------+--------------------+-------+-------+--------------------+


filter column by a single value

In [102]:
df.filter(F.col("name") == "BoJack Horseman").show(5)

+---+---------------+---------------+-------+-------+--------------------+
| id|           name|         genres|runtime| status|        officialSite|
+---+---------------+---------------+-------+-------+--------------------+
|184|BoJack Horseman|[Drama, Comedy]|     25|Running|http://www.netfli...|
+---+---------------+---------------+-------+-------+--------------------+


filter by multiple columns

In [103]:
df.filter(
    (F.col("status") == "Running") & (F.col("runtime") < 45)
).show(5)

+---+------------------+--------------------+-------+-------+--------------------+
| id|              name|              genres|runtime| status|        officialSite|
+---+------------------+--------------------+-------+-------+--------------------+
|142|              Veep|            [Comedy]|     30|Running|http://www.hbo.co...|
|143|    Silicon Valley|            [Comedy]|     30|Running|http://www.hbo.co...|
| 41| Last Man Standing|    [Comedy, Family]|     30|Running|https://www.fox.c...|
| 49|Brooklyn Nine-Nine|[Comedy, Action, ...|     30|Running|https://www.nbc.c...|
| 57|         black-ish|    [Comedy, Family]|     30|Running|http://abc.go.com...|
+---+------------------+--------------------+-------+-------+--------------------+


filter single column by multiple values

In [104]:
df.filter(
    F.col("id").isin(58, 169, 170)
).show(5)

+---+--------------------+--------------------+-------+-------+--------------------+
| id|                name|              genres|runtime| status|        officialSite|
+---+--------------------+--------------------+-------+-------+--------------------+
|169|        Breaking Bad|[Drama, Crime, Th...|     60|  Ended|http://www.amc.co...|
|170|Orange Is the New...|[Drama, Comedy, C...|     60|Running|https://www.netfl...|
| 58|            New Girl|   [Comedy, Romance]|     30|  Ended|http://www.fox.co...|
+---+--------------------+--------------------+-------+-------+--------------------+


filter by string pattern

In [105]:
df.filter(
    F.col("name").like("%BoJack%")
).show(5)

+---+---------------+---------------+-------+-------+--------------------+
| id|           name|         genres|runtime| status|        officialSite|
+---+---------------+---------------+-------+-------+--------------------+
|184|BoJack Horseman|[Drama, Comedy]|     25|Running|http://www.netfli...|
+---+---------------+---------------+-------+-------+--------------------+


In [106]:
df.filter(
    F.col("name").rlike("BoJack|Rick")
).show(5)

+---+---------------+--------------------+-------+-------+--------------------+
| id|           name|              genres|runtime| status|        officialSite|
+---+---------------+--------------------+-------+-------+--------------------+
|184|BoJack Horseman|     [Drama, Comedy]|     25|Running|http://www.netfli...|
|216| Rick and Morty|[Comedy, Adventur...|     30|Running|http://www.adults...|
+---+---------------+--------------------+-------+-------+--------------------+


filter array column by single value

In [107]:
df.filter(
    F.array_contains(F.col("genres"), "Comedy")
).show(5)

+---+------------------+--------------------+-------+-------+--------------------+
| id|              name|              genres|runtime| status|        officialSite|
+---+------------------+--------------------+-------+-------+--------------------+
|128|   Jane the Virgin|[Drama, Comedy, R...|     60|Running|http://www.cwtv.c...|
|129|          Marry Me|            [Comedy]|     30|  Ended|                NULL|
|130|Two and a Half Men|            [Comedy]|     30|  Ended|http://www.cbs.co...|
|131|       About a Boy|[Drama, Comedy, R...|     30|  Ended|http://www.nbc.co...|
|132|       The Millers|            [Comedy]|     30|  Ended|http://www.cbs.co...|
+---+------------------+--------------------+-------+-------+--------------------+


filter array column by multiple values

In [108]:
from functools import reduce
reduce(
    lambda a, b: a.filter(F.array_contains(F.col("genres"), b)),
    ["Comedy", "Crime"],
    df
).show(5, truncate=False)


+---+-----------------------+-----------------------+-------+-------+-----------------------------------------+
|id |name                   |genres                 |runtime|status |officialSite                             |
+---+-----------------------+-----------------------+-------+-------+-----------------------------------------+
|160|Weeds                  |[Drama, Comedy, Crime] |30     |Ended  |http://www.sho.com/sho/weeds/home        |
|170|Orange Is the New Black|[Drama, Comedy, Crime] |60     |Running|https://www.netflix.com/title/70242311   |
|49 |Brooklyn Nine-Nine     |[Comedy, Action, Crime]|30     |Running|https://www.nbc.com/brooklyn-nine-nine   |
|78 |The Mysteries of Laura |[Comedy, Crime]        |60     |Ended  |http://www.nbc.com/the-mysteries-of-laura|
+---+-----------------------+-----------------------+-------+-------+-----------------------------------------+


filter where column is null

In [109]:
df.filter(
    F.col("officialSite").isNull()
).show(5)

+---+-------------+--------------------+-------+------+------------+
| id|         name|              genres|runtime|status|officialSite|
+---+-------------+--------------------+-------+------+------------+
|129|     Marry Me|            [Comedy]|     30| Ended|        NULL|
|134|       Heroes|[Action, Science-...|     60| Ended|        NULL|
|136|The McCarthys|    [Comedy, Family]|     30| Ended|        NULL|
|137|     Cristela|     [Comedy, Legal]|     30| Ended|        NULL|
|147| Nurse Jackie|[Drama, Comedy, T...|     30| Ended|        NULL|
+---+-------------+--------------------+-------+------+------------+


filter where column is not null

In [110]:
df.filter(
    F.col("officialSite").isNotNull()
).show(5)

+---+------------------+--------------------+-------+-------+--------------------+
| id|              name|              genres|runtime| status|        officialSite|
+---+------------------+--------------------+-------+-------+--------------------+
|127|        The Affair|    [Drama, Romance]|     60|Running|http://www.sho.co...|
|128|   Jane the Virgin|[Drama, Comedy, R...|     60|Running|http://www.cwtv.c...|
|130|Two and a Half Men|            [Comedy]|     30|  Ended|http://www.cbs.co...|
|131|       About a Boy|[Drama, Comedy, R...|     30|  Ended|http://www.nbc.co...|
|132|       The Millers|            [Comedy]|     30|  Ended|http://www.cbs.co...|
+---+------------------+--------------------+-------+-------+--------------------+
